# Singleton

Why is the Singleton design pattern useful  
 - Logging — You typically want a single logging object to manage the logging configuration and write logs to a file or another output stream
 - Database Connection — You want to ensure that there is only one instance of the database connection across the application
- Configuration Manager — A configuration manager that reads application settings from a file and provides a global point of access to these settings
- Cache — A caching system where you want a single instance of the cache to be shared across the application

### 1. Using metaclass

In [6]:
class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        print(f'called, with {args=}, {kwargs=}')
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

    # def __call__(cls, *args, **kwargs):
    #     print(f'called, with {args=}, {kwargs=}')
    #     instance = super().__call__(*args, **kwargs)
    #     return instance
        

class MySingleton(metaclass=SingletonMeta):
    def __init__(self, value):
        self.value = value

    def get_value(self):
        return self.value


# Usage
s1 = MySingleton("First instance")
s2 = MySingleton("Second instance") # This will still return the first instance


print(s1 is s2)  # Output: True
print(s1.get_value()) # Output: First instance
print(s2.get_value()) # Output: First instance

called, with args=('First instance',), kwargs={}
called, with args=('Second instance',), kwargs={}
True
First instance
First instance


### 2. Using decorator

In [7]:
def singleton(cls):
    instances = {}
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance


@singleton
class MyDecoratedSingleton:
    def __init__(self, data):
        self.data = data


# Usage
ds1 = MyDecoratedSingleton("Data 1")
ds2 = MyDecoratedSingleton("Data 2")


print(ds1 is ds2) # Output: True
print(ds1.data) # Output: Data 1
print(ds2.data) # Output: Data 1

True
Data 1
Data 1


### 3. Using `__new__` method in a Base Class

In [8]:
class Singleton:
    
    _instance = None
    
    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super().__new__(cls)
        return cls._instance


class MyClass(Singleton):
    
    def __init__(self, a, b=1):
        self.a = a
        self.b = b

    def c(self):
        print(f"{self.a=}, {self.b=}")


cl1 = MyClass(a=1, b=11)
cl2 = MyClass(a=2, b=22)


print(cl1 is cl2) # Output: True

print(cl1.a) # Output: Data 1
print(cl2.a) # Output: Data 1

print(cl1.c())
print(cl2.c())

True
2
2
self.a=2, self.b=22
None
self.a=2, self.b=22
None


### Not thread-safe

In [9]:
class Singleton:
    _instance = None # The class attribute to hold the single instance
    _initialized = False # The class attribute to hold the state

    # The __new__ method is a special method in Python responsible 
    # for creating a new instance of a class
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            # If no instance has been created yet, create one
            cls._instance = super().__new__(cls)
        # Return the single instance
        return cls._instance

    def __init__(self, value):
        if not self._initialized:
            self.value = value
            self._initialized = True


### Thread-safe

In [10]:
import threading

class Singleton:
    _instance = None
    _initialized = False
    _lock = threading.Lock()  # A lock object to synchronize threads

    def __new__(cls, *args, **kwargs):
        # The `with cls._lock` statement is used to acquire the lock before
        # executing the block of code inside it. This ensures that only 
        # one thread can execute this block at a time.
        with cls._lock:
            if cls._instance is None:
                cls._instance = super().__new__(cls)
        return cls._instance

    def __init__(self, value):
        with self._lock:
            if not self._initialized:
                self.value = value
                self._initialized = True

# Try to create two separate instances of the Singleton class
s1 = Singleton(10)
s2 = Singleton(20)

# Check if s1 and s2 are referencing the same object in memory
print(s1 is s2)  # True

# Check the value attribute for s1 and s2
print(s1.value)  # 10
print(s2.value)  # 10

# Another proof that s1 and s2 are referencing the same object in memory
print(s1) # <__main__.Singleton object at 0x75fdc8209430>
print(s2) # <__main__.Singleton object at 0x75fdc8209430>

True
10
10


In [12]:
class Singleton:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super().__new__(cls)
        return cls._instance

    def __init__(self, value):
        self.value = value

s1 = Singleton(10)
s2 = Singleton(20)

print(s1 is s2)  # Output: True (both refer to the same instance)
print(s1.value)  # Output: 10 (value set by the first call to __init__
print(s2.value)  # Output: 10 (value set by the first call to __init__

True
20
20


In [13]:
class A:

    def __init__(self, a, b=1):
        print('called init')
        self.a=a
        self.b=b

    def __new__(cls, *args, **kwargs):
        print('called new')
        print(f'new args: {args}')
        print(f'new kwargs: {kwargs}')
        return super().__new__(cls)

    def __call__(self, value):
        print('called call')
        print(f'call arg: {value}')
        self.a = value

In [14]:
a = A(2, b=3)

called new
new args: (2,)
new kwargs: {'b': 3}
called init


In [15]:
a(123)

called call
call arg: 123


In [16]:
a.a

123